In [1]:
from dataclasses import dataclass

@dataclass
class PreProcessingConfig:
    # Folder directories
    image_root = "/data_vault/hexai/OAICartilage/YOLO_DATASET/Image"
    label_root = "/data_vault/hexai/OAICartilage/YOLO_DATASET/2.5D_Slices_labels"
    out_dir = "/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset"
    dataset_name = "2_5D-Slices"

@dataclass
class YOLOv11Training:
    data_yaml = "/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/KneeMRI.yaml"
    yolo_model = "yolo11n.pt"  # YOLO model for segmentation
    run_name = "SagittalKneeJoint"
    epochs = 50

In [2]:
class KneeLocalizationClasses:
    """
    Map class names for the knee MRI dataset.
    """
    def __init__(self):
        self.classes = {
            0: "SagittalKneeJoint",  # Mapping your class label for the knee joint (adjust as necessary)
        }

    def map_label(self, view):
        """
        Map the class index to a class label. 
        This could dynamically change depending on your dataset.
        """
        return self.classes.get(view, "Unknown")

In [3]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split

def write_yolo_dataset(dataset, image_root, label_root, out_dir, dataset_type):
    """
    Write dataset to the specified directory for training, validation, or testing.
    """
    dataset_outdir = os.path.join(out_dir, dataset_type)
#    dataset_outdir = os.path.join(out_dir, dataset_type)
    os.makedirs(os.path.join(dataset_outdir, "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_outdir, "labels"), exist_ok=True)

    for image_path in dataset:
        filename = os.path.basename(image_path)
        label_filename = filename.replace(".jpg", ".txt")  # Adjust extension if needed
        label_path = os.path.join(label_root, label_filename)

        # Copy image to the output directory
        shutil.copy(image_path, os.path.join(dataset_outdir, "images", filename))

        # Copy label if it exists
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dataset_outdir, "labels", label_filename))

    return dataset_outdir


def build_yolo_dataset(image_root, label_root, out_dir, test_size=0.2):
    os.makedirs(out_dir, exist_ok=True)

    # List all images
    image_files = [os.path.join(subdir, file) for subdir, _, files in os.walk(image_root)
                   for file in files if file.endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp', '.tiff', '.webp'))]

    # Split dataset into train, valid, test
    train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=42)
    valid_files, test_files = train_test_split(test_files, test_size=test_size, random_state=42)

    # Write datasets to their respective directories
    train_path = write_yolo_dataset(train_files, image_root, label_root, out_dir, dataset_type="train")
    valid_path = write_yolo_dataset(valid_files, image_root, label_root, out_dir, dataset_type="valid")
    test_path = write_yolo_dataset(test_files, image_root, label_root, out_dir, dataset_type="test")


    # Write YOLO YAML file
    write_yolo_yaml_file(train_path, valid_path, test_path, out_dir)


In [4]:
def write_yolo_yaml_file(train_path, valid_path, test_path, out_dir):
    class_labels = KneeLocalizationClasses()
    yaml_info = {
        "train": train_path,
        "val": valid_path,
        "test": test_path,
        "nc": 1,  # Number of classes (1 in your case)
        "names": [class_labels.map_label(0)]  # Using 0 as the class index for "SagittalKneeJoint"
    }

    with open(f"{out_dir}/KneeMRI.yaml", 'w') as f:
        yaml.dump(yaml_info, f)


In [5]:
from ultralytics import YOLO

def get_yolo_model(yolo_model_checkpoint):
    """
    YOLOv11 Checkpoints:
    - yolov11n.pt (Nano)
    - yolov11s.pt (Small)
    - yolov11m.pt (Medium)
    - yolov11l.pt (Large)
    - yolov11x.pt (Extra Large)
    """
    return YOLO(yolo_model_checkpoint)


In [6]:
import torch
import os

def train_yolo_model(data_yaml, yolo_checkpoint_name, epochs, run_name):
    model = get_yolo_model(yolo_checkpoint_name)

    if torch.cuda.is_available():
        model = model.cuda()

    # Check if the YAML file exists
    if os.path.exists(data_yaml):
        print("data.yaml found. Proceeding with training.")
        model.train(data=data_yaml, epochs=epochs, name=run_name)
    else:
        print("data.yaml not found. Please check the path and ensure the file exists.")


In [7]:
def main(mode):
    print("Knee MRI Segmentation Entry Point!")

    if mode == "preprocess":
        config = PreProcessingConfig()
        build_yolo_dataset(config.image_root, config.label_root, config.out_dir)
        print("Preprocessing completed!")
    
    elif mode == "yolo":
        config = YOLOv11Training()
        train_yolo_model(config.data_yaml, config.yolo_model, config.epochs, config.run_name)
        print("Training completed!")


In [ ]:
if __name__ == "__main__":
    main("preprocess")  # Run preprocessing

In [ ]:
main("yolo")  # Run YOLO model training

In [ ]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('/data_vault/hexai/OAICartilage/runs/detect/SagittalKneeJoint/weights/best.pt')
# Evaluate on the validation set
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
# Evaluate on the test set
metrics = model.val(data="/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/KneeMRI.yaml", split="test") # This will give precision, recall, mAP, and other relevant metrics

In [ ]:
# Print the metrics for detailed evaluation
print("Validation Metrics:", metrics)